In [1]:
import json
import pickle

import numpy as np
# import tensorflow_hub as hub
# import tensorflow as tf

from tqdm import tqdm
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frizman04\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
with open('data/corrected_tables_v2.json','r') as f:
    data = json.load(f)
   
print(len(data))

18


In [0]:
elma_path = r"http://files.deeppavlov.ai/deeppavlov_data/oil-gas_epoches_n_13.tar.gz"
# elma_path = r'model/oil-gas_epoches_n_13'

elmo = hub.Module(elma_path, trainable=False)

In [0]:
init = tf.global_variables_initializer()
table_vect_dict = dict()

for table_indx,table in tqdm(enumerate(data)) :
    
    tokens_input = []
    for row_indx, row in enumerate(table['rows']) :
        for col_indx,col_name in enumerate(table['columns']) :

            cell = row[col_indx]        
            row_name = ''
            if table['name_entity_col'] != None :
                row_name = table['rows'][row_indx][table['name_entity_col']]

            cell_full = " ".join([col_name, row_name, cell])

            tokens = word_tokenize(cell_full.lower())
            tokens_input.append(tokens)

    tokens_length = [len(tok) for tok in tokens_input]

    #заполним до максимума
    tokens_length_max = max(tokens_length)
    tmp = []
    for tok in tokens_input :
        tmp.append(tok + ["" for _ in range(tokens_length_max - len(tok))] )
    tokens_input = tmp
    
    
    #get embed from elma
    feed_dict = {"tokens": tokens_input,"sequence_len": tokens_length}
    embeddings = elmo(inputs=feed_dict,signature="tokens", as_dict=True)["elmo"]
    print('Calc embed : %s' % embeddings)

    #calc embeddings
    with tf.Session() as sess :
        sess.run(init)
        table_vect = embeddings.eval(session=sess)
        
    print('Shape of emed : %s ' % str(table_vect.shape))

    #reshape table vector
    table_vect = table_vect.mean(axis=1)
    rows_len = len(table['rows'])
    col_len = len(table['columns'])
    table_vect = table_vect.reshape((rows_len,col_len,table_vect.shape[1]))

    #save result
    table_vect_dict[table_indx] = table_vect
#     pickle.dump(table_vect,open(r'tmp/table_vec_' + str(table_indx) +'.p','wb'))

In [0]:
with open('table_vect_dict.p','wb') as f :
  pickle.dump(table_vect_dict,f)
  
print(len(table_vect_dict))


# Calc accuracy

In [1]:
import numpy as np

from tqdm import tqdm
import pandas as pd
import pickle

import importlib
import table_retrival
importlib.reload(table_retrival)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frizman04\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frizman04\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'table_retrival' from 'D:\\Programing\\DataMonsters\\Tabels\\Table retrive\\experimental\\table_retrival.py'>

In [2]:
df = pd.read_table("data/questions_v2_answers.csv",sep=',')
print(len(df))
df.head()

38


,table_index,question,answer,Unnamed: 3
0,0,What is the feedstock for Sulphur recovery for...,22,NaN
1,0,Which component has a minimum Sulphur Recover...,Hydrocarbon (as C1),NaN
2,1,What is the consumed power in Sulphur Recovery...,83 kW,NaN
3,2,What is the total feedstock for Sulphur recove...,2167,NaN
4,3,What is an energy balance for Acid Gas Preheat...,-3994,NaN


In [3]:
y_head = pickle.load(open('tmp/y_head_row_withtitleheader_q_in_lower.p','rb'))

In [7]:
# is_correct = []
# for indx in range(len(df['answer'])) :
#     correct_answer = df.loc[indx]['answer']
    
#     print('Correct answer : %s' % correct_answer)
#     for indx_prop,prop_answer in enumerate(y_head[indx]['answer']) :
#         print("%s %s" % (indx_prop,prop_answer))
        
#     is_correct.append(input())
#     print('--------------------------------------------------------------------')
# is_correct = [int(i) for i in is_correct]



# is_correct = [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
# acc = (sum(is_correct) / len(is_correct)) * 100
# print(acc)


# is_correct = [0, 4, 0, 5, 9, 9, 10, 9, 10, 2, 10, 5, 1, 0, 8, 0, 0, 0, 1, 1, 1, 8, 8, 10, 1, 2, 0, 3, 0, 1, 4, 8, 10, 10, 1, 0, 1, 7]
# is_correct =[0, 0, 10, 4, 10, 7, 10, 6, 1, 0, 10, 0, 2, 2, 10, 0, 0, 2, 1, 0, 3, 9, 9, 1, 1, 2, 1, 3, 1, 2, 10, 3, 8, 8, 2, 1, 0, 0]
# is_correct =  [0, 0, 2, 4, 10, 1, 5, 5, 1, 0, 10, 0, 9, 1, 10, 0, 0, 1, 4, 1, 2, 8, 8, 10, 1, 4, 3, 2, 2, 2, 5, 1, 8, 2, 3, 10, 1, 0]

# is_correct = [0, 4, 10, 3, 10, 10, 3, 6, 3, 0, 8, 0, 0, 1, 10, 0, 0, 3, 0, 0, 3, 7, 10, 10, 1, 2, 1, 3, 3, 3, 10, 4, 2, 4, 0, 1, 0, 0]
# # is_correct = [0, 7, 0, 5, 10, 10, 10, 1, 0, 0, 8, 4, 3, 10, 10, 1, 0, 3, 10, 0, 1, 9, 7, 10, 4, 3, 3, 2, 3, 0, 3, 0, 10, 0, 2, 4, 0, 0]

is_correct = [0, 4, 10, 3, 10, 10, 3, 6, 3, 0, 8, 0, 0, 1, 10, 0, 0, 10, 0, 0, 3, 7, 10, 10, 1, 2, 1, 3, 1, 1, 10, 4, 2, 4, 0, 1, 0, 0]
is_correct = np.array(is_correct)

In [61]:
# print(is_correct)

[0, 7, 0, 5, 10, 10, 10, 1, 0, 0, 8, 4, 3, 10, 10, 1, 0, 3, 10, 0, 1, 9, 7, 10, 4, 3, 3, 2, 3, 0, 3, 0, 10, 0, 2, 4, 0, 0]


In [9]:
for n in range(1,10) :
    rec_n = np.sum(is_correct <= (n - 1)) / len(is_correct)
    print('recall for %.f = %.4f' % (n,rec_n))

recall for 1 = 0.2895
recall for 2 = 0.4474
recall for 3 = 0.5000
recall for 4 = 0.6316
recall for 5 = 0.7105
recall for 6 = 0.7105
recall for 7 = 0.7368
recall for 8 = 0.7632
recall for 9 = 0.7895


# Error analisis

In [15]:
df = pd.read_table("data/questions_v2_answers.csv",sep=',')
df.head()

,table_index,question,answer,Unnamed: 3
0,0,What is the feedstock for Sulphur recovery for...,22,NaN
1,0,Which component has a minimum Sulphur Recover...,Hydrocarbon (as C1),NaN
2,1,What is the consumed power in Sulphur Recovery...,83 kW,NaN
3,2,What is the total feedstock for Sulphur recove...,2167,NaN
4,3,What is an energy balance for Acid Gas Preheat...,-3994,NaN


In [16]:
y_head = pickle.load(open('tmp/y_head.p','rb'))
print(y_head)

{0: {'answer': ['22\n', 'Hydrocarbon (as C1) ', '', '1% ', 'H2S ', '60% ', '1300\n', '195\n', '9% ', 'H2O '], 'cell_index': array([[[2, 3],
        [2, 0],
        [2, 2],
        [2, 1],
        [0, 0],
        [0, 1],
        [0, 3],
        [3, 3],
        [3, 1],
        [3, 0]]], dtype=int64), 'cos_sim': array([0.5471239 , 0.5260725 , 0.50955176, 0.50876915, 0.44930673,
       0.4349506 , 0.4298541 , 0.41842636, 0.4150135 , 0.4066299 ],
      dtype=float32)}, 1: {'answer': ['Hydrocarbon (as C1) ', '22\n', 'H2S ', 'Total ', '', '1% ', 'H2O ', 'CO2 ', '100% ', '60% '], 'cell_index': array([[[2, 0],
        [2, 3],
        [0, 0],
        [4, 0],
        [2, 2],
        [2, 1],
        [3, 0],
        [1, 0],
        [4, 1],
        [0, 1]]], dtype=int64), 'cos_sim': array([0.43921617, 0.42403573, 0.42225587, 0.41432372, 0.40989518,
       0.40604934, 0.38650203, 0.38152203, 0.36774883, 0.3671286 ],
      dtype=float32)}, 2: {'answer': ['N/A ', '12,266 kW ', '18,610 kW ', 'N/A ', '86

      dtype=float32)}}


In [17]:
tmp = []
for question_index in range(len(df)) :
    
    table_index = df.loc[question_index]['table_index']
    correct_answer = df.loc[question_index]['answer']
    question =  df.loc[question_index]['question']
    
    for pos_answer,cos_sim in zip(y_head[question_index]['answer'],y_head[question_index]['cos_sim']) :
        tmp.append([table_index,question,correct_answer,pos_answer,cos_sim])
        
df_new = pd.DataFrame(tmp,columns=['table_index','question','correct_answer','possible_answer','cos_sim'])
df_new

df_new.to_csv('data/questions_v2_answers_error_analysis.csv')